In [ ]:
!pip install tweepy

In [13]:
import tweepy
import pandas as pd
import time
import re
import numpy as np
import json

# Connecting to Twitter API

In [14]:
credentials = {}
with open("twitter_credentials.txt") as f:
    for line in f:
       (key, val) = line.split(':')
       credentials[key] = val

In [15]:
consumer_key = credentials['consumer_key']
consumer_secret = credentials['consumer_secret']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [207]:
def extract_timeline_as_df(timeline_list,search_word,date_since,n_results):
    columns = set()
    allowed_types = [str, int,bool]
    tweets_data = []
    for status in tweepy.Cursor(api.search,q=search_words, lang="es", tweet_mode='extended').items(n_results):        
        try:
            status_dict = dict(vars(status))
            keys = status_dict.keys()
            single_tweet_data = {"author": status.author.screen_name,"author_id":str(status.author.id)}
            for k in keys:
                try:
                    v_type = type(status_dict[k])
                except:
                    v_type = None
                if v_type != None:
                    if v_type in allowed_types:
                        single_tweet_data[k] = status_dict[k]
                        columns.add(k)
            if 'retweeted_status' in keys:
                single_tweet_data['full_text'] = status.retweeted_status.full_text
            tweets_data.append(single_tweet_data)
        except:
            continue
    header_cols = list(columns)
    header_cols.append('author')
    header_cols.append("author_id")
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [26]:
search_words = "mujeres+videojuegos OR chicas+videojuegos OR tias+videojuegos"
date_since = "2018-11-16"

In [221]:
df = extract_timeline_as_df(api.home_timeline(),s,date_since,1000)

In [68]:
df = pd.read_csv('tweetsDF.csv')
df

,Unnamed: 0,quoted_status_id,quoted_status_id_str,lang,possibly_sensitive,id_str,id,truncated,in_reply_to_status_id_str,source,...,source_url,retweet_count,in_reply_to_user_id_str,in_reply_to_user_id,is_quote_status,in_reply_to_status_id,favorited,favorite_count,author,author_id
0,0,NaN,NaN,es,NaN,1332662039328731138,1332662039328731138,False,NaN,Twitter for iPhone,...,http://twitter.com/download/iphone,4,NaN,NaN,False,NaN,False,0,carmen4366,2875077087
1,1,NaN,NaN,es,NaN,1332656127864139782,1332656127864139782,False,NaN,Twitter for Android,...,http://twitter.com/download/android,0,NaN,NaN,False,NaN,False,2,ansexder,1193667502301237248
2,2,NaN,NaN,es,NaN,1332639420701401089,1332639420701401089,False,1.332333e+18,Twitter Web App,...,https://mobile.twitter.com,0,1.631909e+09,1.631909e+09,False,1.332333e+18,False,0,Sebatian2840,2351349126
3,3,NaN,NaN,es,NaN,1332632576771633153,1332632576771633153,False,1.332633e+18,Twitter for Android,...,http://twitter.com/download/android,0,8.192529e+17,8.192529e+17,False,1.332633e+18,False,0,Limon_Kawaii,819252867991240705
4,4,NaN,NaN,es,NaN,1332626228822630403,1332626228822630403,False,1.332626e+18,Twitter Web App,...,https://mobile.twitter.com,0,4.595703e+08,4.595703e+08,False,1.332626e+18,False,0,nemuritor98,459570327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,585,NaN,NaN,es,NaN,1329434790957883396,1329434790957883396,False,1.329434e+18,Twitter Web App,...,https://mobile.twitter.com,0,7.018225e+07,7.018225e+07,False,1.329434e+18,False,1,nucaica,369664862
274,586,NaN,NaN,es,NaN,1329432613447217157,1329432613447217157,False,1.329433e+18,Twitter for Android,...,http://twitter.com/download/android,0,3.044738e+09,3.044738e+09,False,1.329433e+18,False,1,Pedrojose_113,3044738452
275,587,NaN,NaN,es,NaN,1329409221226606593,1329409221226606593,False,1.329409e+18,Twitter Web App,...,https://mobile.twitter.com,0,7.536650e+17,7.536650e+17,False,1.329409e+18,False,1,JosRV2000,753665009830531072
276,588,NaN,NaN,es,False,1329402337396482050,1329402337396482050,False,1.329400e+18,Twitter Web App,...,https://mobile.twitter.com,0,3.313058e+09,3.313058e+09,False,1.329400e+18,False,0,ObsNoctambulas,3313057823


In [214]:
df.drop_duplicates('full_text',inplace =True)

In [215]:
df

,quoted_status_id,quoted_status_id_str,lang,possibly_sensitive,id_str,id,truncated,in_reply_to_status_id_str,source,in_reply_to_screen_name,...,source_url,retweet_count,in_reply_to_user_id_str,in_reply_to_user_id,is_quote_status,in_reply_to_status_id,favorited,favorite_count,author,author_id
0,NaN,NaN,es,NaN,1333045112184709121,1333045112184709121,False,1333043996273348609,Twitter for Android,Karrizu_,...,http://twitter.com/download/android,0,587737603,587737603.0,False,1.333044e+18,False,0,Karrizu_,587737603
1,1.332958e+18,1332958223330992129,es,NaN,1333043749350420480,1333043749350420480,False,NaN,Twitter for Android,NaN,...,http://twitter.com/download/android,4,NaN,NaN,True,NaN,False,0,Sandychan_n,220345519
2,NaN,NaN,es,NaN,1333041270902710272,1333041270902710272,False,NaN,Twitter for Android,NaN,...,http://twitter.com/download/android,8,NaN,NaN,False,NaN,False,0,alebarbiie,27141348
4,NaN,NaN,es,False,1333026095780782080,1333026095780782080,False,NaN,Twitter for Android,NaN,...,http://twitter.com/download/android,0,NaN,NaN,False,NaN,False,0,marescrack,3204688732
5,NaN,NaN,es,NaN,1333012241118081034,1333012241118081034,False,1333008180012572672,Twitter for iPhone,obsesiones21,...,http://twitter.com/download/iphone,0,57923284,57923284.0,False,1.333008e+18,False,4,soris_dissastr,1049076838042099712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,1.329722e+18,1329721722174656512,es,False,1329723914436694016,1329723914436694016,False,NaN,Twitter for iPhone,NaN,...,http://twitter.com/download/iphone,0,NaN,NaN,True,NaN,False,2,cuquiquinn,1232808377417371649
605,NaN,NaN,es,NaN,1329720058025816065,1329720058025816065,False,1329714284218212354,Twitter for Android,TitoDeadpooloso,...,http://twitter.com/download/android,0,107815551,107815551.0,False,1.329714e+18,False,0,Althaell,2998511224
607,NaN,NaN,es,NaN,1329689523006156800,1329689523006156800,False,1329574725836812289,Twitter for Android,stivenmorales95,...,http://twitter.com/download/android,0,177272685,177272685.0,False,1.329575e+18,False,2,2606Noelia,1166708502976434178
608,NaN,NaN,es,False,1329686947875483651,1329686947875483651,False,NaN,Twitter for iPhone,NaN,...,http://twitter.com/download/iphone,0,NaN,NaN,False,NaN,False,0,FOlalquiaga,1081223882311430149


In [12]:
df.to_csv('tweetsDF.csv')

def clean_text_column(text):
    text = text.replace('RT','')
    text = re.sub(r"(?:\@|https?\://)\S+", "", text).strip()
    return text

## TAGS

In [6]:
games = ['juegos','videojuegos','gaming','competiciones','streaming']
fem = ['jugadoras','mujeres','chicas','feminazi','machismo','feminismo']
combinations = [game+','+f for game in games for f in fem]

In [243]:
combinations

['juegos,jugadoras',
 'juegos,mujeres',
 'juegos,chicas',
 'juegos,feminazi',
 'juegos,machismo',
 'juegos,feminismo',
 'videojuegos,jugadoras',
 'videojuegos,mujeres',
 'videojuegos,chicas',
 'videojuegos,feminazi',
 'videojuegos,machismo',
 'videojuegos,feminismo',
 'gaming,jugadoras',
 'gaming,mujeres',
 'gaming,chicas',
 'gaming,feminazi',
 'gaming,machismo',
 'gaming,feminismo',
 'competiciones,jugadoras',
 'competiciones,mujeres',
 'competiciones,chicas',
 'competiciones,feminazi',
 'competiciones,machismo',
 'competiciones,feminismo',
 'streaming,jugadoras',
 'streaming,mujeres',
 'streaming,chicas',
 'streaming,feminazi',
 'streaming,machismo',
 'streaming,feminismo']

## Creating a stream listener

In [3]:
class MyStreamListener(tweepy.StreamListener):

    def __init__(self, api=None):
        # inherit class attributes
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets2.txt", "a")
        
    def on_error(self, status_code):
        if status_code == 420:
            print('420 error')
            #Ends stream in case of rate limiting
        print(status_code)
        return False 
        
    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + ',\n' )
        
        self.num_tweets += 1
        if self.num_tweets < 5000:
            return True
        else:
            return False
        self.file.close()   

In [4]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [7]:
myStream.filter(languages=["es"],track=games+fem, is_async=True)

## Agregate obtained values to DF

In [176]:
with open('improved_tweets.json') as json_file:
    data = json.load(json_file)
    columns = set()
    allowed_types = [str, int,bool]
    tweets_data = []
    for status in data:
        status = tweepy.Status().parse(None, status)
        try:
            status_dict = dict(vars(status))
            keys = status_dict.keys()
            single_tweet_data = {"author": status_dict['_json']['user']['screen_name'],"author_id":status_dict['_json']['user']['id_str']}
            for k in keys:
                try:
                    v_type = type(status_dict[k])
                except:
                    v_type = None
                if v_type != None:
                    if v_type in allowed_types:
                        single_tweet_data[k] = status_dict[k]
                        columns.add(k)
            if 'retweeted_status' in keys:
                single_tweet_data['full_text'] = status.retweeted_status.full_text
            tweets_data.append(single_tweet_data)
        except :
            continue
header_cols = list(columns)
header_cols.append('author')
header_cols.append("author_id")
df2 = pd.DataFrame(tweets_data, columns=header_cols)
df2

     quoted_status_id quoted_status_id_str lang possibly_sensitive  \
0                 NaN                  NaN   en                NaN   
1                 NaN                  NaN   es                NaN   
2                 NaN                  NaN  und              False   
3                 NaN                  NaN   es                NaN   
4                 NaN                  NaN   es                NaN   
..                ...                  ...  ...                ...   
437               NaN                  NaN   es              False   
438               NaN                  NaN   en               True   
439               NaN                  NaN   es              False   
440               NaN                  NaN   en               True   
441               NaN                  NaN   es              False   

                  id_str  reply_count                   id   timestamp_ms  \
0    1332749776744574976            0  1332749776744574976  1606587282726   
1    

In [178]:
df3 = pd.concat([df,df2])

In [179]:
df3

,Unnamed: 0,quoted_status_id,quoted_status_id_str,lang,possibly_sensitive,id_str,id,truncated,in_reply_to_status_id_str,source,...,in_reply_to_status_id,favorited,favorite_count,author,author_id,reply_count,timestamp_ms,quote_count,filter_level,text
0,0.0,NaN,NaN,es,NaN,1332662039328731138,1332662039328731138,False,NaN,Twitter for iPhone,...,NaN,False,0,carmen4366,2875077087,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,es,NaN,1332656127864139782,1332656127864139782,False,NaN,Twitter for Android,...,NaN,False,2,ansexder,1193667502301237248,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,es,NaN,1332639420701401089,1332639420701401089,False,1.33233e+18,Twitter Web App,...,1.332333e+18,False,0,Sebatian2840,2351349126,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,es,NaN,1332632576771633153,1332632576771633153,False,1.33263e+18,Twitter for Android,...,1.332633e+18,False,0,Limon_Kawaii,819252867991240705,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,es,NaN,1332626228822630403,1332626228822630403,False,1.33263e+18,Twitter Web App,...,1.332626e+18,False,0,nemuritor98,459570327,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,NaN,NaN,NaN,es,False,1332808982264737793,1332808982264737793,False,NaN,Twitter for Android,...,NaN,False,0,Bahamut1892,1491049057,0.0,1606601398422,0.0,low,La buena cena 😁 https://t.co/PvCcmAyClJ
438,NaN,NaN,NaN,en,True,1332809104201576450,1332809104201576450,False,NaN,PlayStation®Network,...,NaN,False,0,kryznr,1323261102936018944,0.0,1606601427494,0.0,low,Joinnn me live at https://t.co/vT9Xtbl3SR
439,NaN,NaN,NaN,es,False,1332809131397361665,1332809131397361665,False,NaN,Twitter for Android,...,NaN,False,0,Astraxolotlgirl,753390369019039744,0.0,1606601433978,0.0,low,Yaaaaaa está @mbonfil_65 póngale a su canal 🥰...
440,NaN,NaN,NaN,en,True,1332809104201576450,1332809104201576450,False,NaN,PlayStation®Network,...,NaN,False,0,kryznr,1323261102936018944,0.0,1606601427494,0.0,low,Joinnn me live at https://t.co/vT9Xtbl3SR


In [130]:
t = tweepy.models.Status(data[0])

In [171]:
stat = tweepy.Status().parse(None, data[0])

tweepy.Status().parse(None, data[0])

In [172]:
status_dict = dict(vars(stat))
keys = status_dict.keys()

# Getting tweets from file txt

In [17]:
from pandas.io.json import json_normalize
i = 0
dfflat = pd.DataFrame()
with open('sela-latin-america-misogyny.txt') as json_file:
    data = json.load(json_file)
for tweet in data:
    i+=1
    if i > 10000:
        break
    df_for_tweet = json_normalize(tweet)
    dfflat=dfflat.append(df_for_tweet,ignore_index=True,sort=True)

JSONDecodeError: Extra data: line 2 column 1 (char 20)